# YOLO fine tuning by YOLO library #

### I used [Traffic Signs Dataset in YOLO format](https://www.kaggle.com/datasets/valentynsichkar/traffic-signs-dataset-in-yolo-format) ###

In [1]:
import os
import random
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.103  Python-3.9.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Setup complete  (6 CPUs, 15.9 GB RAM, 572.6/931.5 GB disk)


### Prepare dataset to fine-tune ###

In [37]:
#specify paths to folders
train_path = 'some_folder\\train'
valid_path = 'some_folder\\valid'
test_path = 'some_folder\\test'

In [ ]:
ano_paths=[]

for dirname, _, filenames in os.walk('dataset\\ts\\ts'):
    for filename in filenames:
        ano_paths+=[(os.path.join(dirname, filename))]

# ano_paths = [i.replace('\\', '/') for i in ano_paths]
n=len(ano_paths)
N=list(range(1,len(ano_paths),2))
random.shuffle(N)

valid_count = 100
test_count = 50


test_i = N[:test_count]
train_i = N[test_count:-valid_count]
valid_i = N[-valid_count:]

In [46]:
def create_dataset(list_of_pictures, path):
    for i in list_of_pictures:
        ano_path=ano_paths[i]

        img_path = os.path.join('dataset\\ts\\ts', ano_path.split('\\')[-1][0:-4]+'.jpg')
        try:
            !copy {ano_path} {path} #use cp on linux
            !copy {img_path} {path} #use cp on linux
        except:
            continue

In [ ]:
create_dataset(train_i, train_path)
create_dataset(valid_i, valid_path)
create_dataset(test_i, test_path)

### Create .yaml file ###

In [54]:
import yaml

data_yaml = dict(
    train = "some_folder\\train",
    val = "some_folder\\valid",
    test= "some_folder\\test",
    nc = 4,
    names = ['prohibitor','danger','mandatory','other']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

### Start fine-tuning ###
find in output 'Results saved to ...'

In [ ]:
model = YOLO("yolov8x.pt")
#wandb.login() or run in terminal
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=12 imgsz=480

### Predict data ###

In [ ]:
model_path = "output\\weights\\best.pt"
source_path = "dataset\\traffic-sign-to-test.mp4"
!yolo task=detect mode=predict model={model_path} conf=0.5 source={source_path}